# Finding Lane Lines on the Road by Tao Liang (lotus)


## Project: **Finding Lane Lines on the Road** 
***
In this project, I will use the tools have been learned about in the lesson to identify lane lines on the road.

## Import Packages

In [56]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Lane Detection Pipeline

**Pipeline:**

`Color select` for white or yellow selection  
`Grayscale` for color transform  
`Smoothing: Gaussian Blur`  
`Edges Detect Through Canny`
`ROI region interest`
`Hough Line Detect`  
`Blend line image into origin image`

## Helper Functions

Below are some helper functions copied from Lesson. And I add some new functions for helpler such as color_select_rgb_white_yellow, region_vertices, save_image and draw_lane_lines.

In [86]:
import math

def read_image(file):
    """Just read an image file
    """
    return mpimg.imread(file)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def color_select_rgb_white_yellow(image):
    """Select color white or yellow
    This will return an image with only white or yellow colors
    """
    lower = np.uint8([200, 200, 200])
    upper = np.uint8([255, 255, 255])

    color_white = cv2.inRange(image, lower, upper)

    lower = np.uint8([190, 190, 0])
    upper = np.uint8([255, 255, 255])

    color_yellow = cv2.inRange(image, lower, upper)
    mask = cv2.bitwise_or(color_white, color_yellow)
    masked = cv2.bitwise_and(image, image, mask=mask)
    return masked

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_vertices(image):
    """
    Auto compute region vertices with a stable window [(0.1, -2), (0.45, 0.6), (0.6, 0.6), (0.1, -2)]
    """
    width = image.shape[1]
    height = image.shape[0]

    v_top_left = [int(width*0.45), int(height*0.6)]
    v_top_right = [int(width*0.55), int(height*0.6)]
    v_bottom_left = [int(width*0.05), height]
    v_bottom_right = [int(width*0.95), height]
    return np.array([[v_bottom_left, v_top_left, v_top_right, v_bottom_right]], dtype=np.int32)

def region_of_interest(img, vertices=None):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    if vertices is None:
        vertices = region_vertices(img)
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def save_image(img, file):
    """Save image into a specified file
    """
    cv2.imwrite(file, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def average_slope_intercept(lines):
    left_lane_lines    = [] # (slope, intercept)
    left_lane_weights  = [] # (length,)
    right_lane_lines   = [] # (slope, intercept)
    right_lane_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if length < 10:
                continue
            if slope < 0: # y is reversed in image
                left_lane_lines.append((slope, intercept))
                left_lane_weights.append((length))
            else:
                right_lane_lines.append((slope, intercept))
                right_lane_weights.append((length))

    # Weight slopes and Y_intercepts by their line lenght
    right_lane = np.dot(right_lane_weights, right_lane_lines) / np.sum(right_lane_weights) if len(right_lane_weights) > 0 else None
    left_lane  = np.dot(left_lane_weights,  left_lane_lines) / np.sum(left_lane_weights)  if len(left_lane_weights) > 0 else None

    return right_lane, left_lane

def make_points(y1, y2, line):
    if line is None:
        return None

    slope, intercept = line
    
    if not slope:
        return None

    x1 = int((y1 - intercept)/slope)
    y1 = int(y1)
    x2 = int((y2 - intercept)/slope)
    y2 = int(y2)

    return ((x1, y1), (x2, y2))

def make_lane_lines(image, lines):
    right_lane, left_lane = average_slope_intercept(lines) # seperate lane line into right lane and left lane

    y1 = image.shape[0] # Y value Opencv has everything inverted
    y2 = y1 * 0.60

    right_line = make_points(y1, y2, right_lane)
    left_line  = make_points(y1, y2, left_lane)

    return right_line, left_line

def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=2):
    line_img = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    for line in lines:
        if line:
            cv2.line(line_img, *line, color, thickness)
    return weighted_img(image, line_img)

## Test Images

Build pipeline to work on the images in the directory "test_images" and generate outputs in the directory "test_images_output".

In [82]:
import os
files = []
output_files = []
if not os.path.exists("test_images_output/"):
    os.mkdir("test_images_output/")

for file in os.listdir("test_images/"):
    if '.jpg' in file:
        files.append(os.path.join("test_images/", file))
        output_files.append(os.path.join("test_images_output/", file))

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [87]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.
from IPython.display import display, HTML
# PIPELINE: Color Select -> RGB2Gray -> Gaussian Blur Smoothing -> Edge Detect through Canny -> ROI Select -> Line Detect -> Draw Lines
def save_images(images, prefix=''):
    for i in range(len(images)):
        cv2.imwrite(os.path.join("test_images_output/", "%s_image_%02d.jpg" % (prefix, i)), cv2.cvtColor(images[i], cv2.COLOR_RGB2BGR))

# Load images
origin_images = list(map(read_image, files))
# Color select
white_yellow = list(map(color_select_rgb_white_yellow, origin_images))
save_images(white_yellow, 'white_yellow')
# RGB2GRAY
gray_images = list(map(grayscale, white_yellow))
save_images(gray_images, 'grayscale')
# Smoothing: Gaussian Blur
smoothing_gaussian = list(map(lambda image:gaussian_blur(image, 5), gray_images))
save_images(smoothing_gaussian, 'blur')
# Edges Detect Through Canny
edges_canny = list(map(lambda image: canny(image, 50, 150), smoothing_gaussian))
save_images(edges_canny, 'edge')
# ROI
roi_regions = list(map(region_of_interest, edges_canny))
save_images(roi_regions, 'roi')
# Line Detect
all_line_images = list(map(lambda image: hough_lines(image, rho=2, theta=math.pi/180, threshold=10, min_line_len=10, max_line_gap=10), roi_regions))
# Blend line image into origin image
for i in range(len(all_line_images)):
    combo = weighted_img(all_line_images[i][0], origin_images[i])
    save_image(combo, output_files[i])
    right_line, left_line = make_lane_lines(all_line_images[i][0], all_line_images[i][1])
    line_combo = draw_lane_lines(origin_images[i], (right_line, left_line), thickness=18)
    save_image(line_combo, "test_images_output/extrapolate_line_%02d.jpg" % (i))

In [88]:
for i in range(6):
    display(HTML("""
    <img src="test_images_output/extrapolate_line_0{0}.jpg" width="480"/>
    """.format(i)))

## Test on Videos

I test mine solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [89]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [90]:
from collections import deque
class LaneLineFinder:

    SAMPLE_FRAMES = 10

    def __init__(self):
        self.right_lane_lines = deque(maxlen= self.SAMPLE_FRAMES)
        self.left_lane_lines = deque(maxlen= self.SAMPLE_FRAMES)

    def average_line_sampling(self, line, previous_lines):
        if line:
            previous_lines.append(line)

        if len(previous_lines) > 0:
            line = np.mean(previous_lines, axis = 0, dtype=np.int32) # Like tensorflow
            line = tuple(map(tuple, line))

        return line
    
    def process_image(self, image, extrapolate=True):
        # Color select
        white_yellow = color_select_rgb_white_yellow(image)
        # RGB2GRAY
        gray_image = grayscale(white_yellow)
        # Smoothing: Gaussian Blur
        smoothing_gaussian = gaussian_blur(gray_image, 5)
        # Edges Detect Through Canny
        edges_canny = canny(smoothing_gaussian, 50, 150)
        # ROI
        roi_region = region_of_interest(edges_canny)
        # Line Detect
        line_image, lines = hough_lines(roi_region, rho=1, theta=math.pi/180, threshold=20, min_line_len=20, max_line_gap=300)
        # Extrapolate lanelines return by the Hough transform
        extrapolated_lane_lines = make_lane_lines(image, lines)

        # Average the videos last 6 lines' frames
        right_line = self.average_line_sampling(extrapolated_lane_lines[0], self.right_lane_lines)
        left_line = self.average_line_sampling(extrapolated_lane_lines[1], self.left_lane_lines)
        if extrapolate is True:
            return draw_lane_lines(image, (right_line, left_line), thickness=18)
        # Blend line image into origin image
        else:
            combo = weighted_img(line_image, image)
            return combo

Let's try the one with the solid white lane on the right first ...

In [91]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
lineFinder = LaneLineFinder()
white_clip = clip1.fl_image(lineFinder.process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4




  0%|          | 0/222 [00:00<?, ?it/s]

  2%|▏         | 4/222 [00:00<00:05, 39.07it/s]

  5%|▍         | 10/222 [00:00<00:04, 43.41it/s]

  7%|▋         | 16/222 [00:00<00:04, 46.70it/s]

 10%|▉         | 22/222 [00:00<00:04, 48.73it/s]

 13%|█▎        | 28/222 [00:00<00:03, 50.66it/s]

 16%|█▌        | 35/222 [00:00<00:03, 53.63it/s]

 19%|█▉        | 42/222 [00:00<00:03, 55.76it/s]

 22%|██▏       | 48/222 [00:01<00:04, 37.24it/s]

 24%|██▍       | 53/222 [00:01<00:04, 34.68it/s]

 26%|██▌       | 57/222 [00:01<00:05, 29.32it/s]

 27%|██▋       | 61/222 [00:01<00:06, 25.98it/s]

 29%|██▉       | 65/222 [00:01<00:06, 25.07it/s]

 31%|███       | 68/222 [00:01<00:06, 25.56it/s]

 32%|███▏      | 71/222 [00:02<00:06, 25.12it/s]

 33%|███▎      | 74/222 [00:02<00:06, 22.72it/s]

 35%|███▍      | 77/222 [00:02<00:05, 24.22it/s]

 36%|███▌      | 80/222 [00:02<00:05, 24.23it/s]

 37%|███▋      | 83/222 [00:02<00:05, 23.68it/s]

 39%|███▊      | 86/222 [00:02<00:06, 22.52it/s]

 40%|███

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 3.01 s, sys: 209 ms, total: 3.22 s
Wall time: 9.58 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [92]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**Extrapolate lines please see function `draw_lane_lines`**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [93]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
lineFinder = LaneLineFinder()
yellow_clip = clip2.fl_image(lambda image: lineFinder.process_image(image, True))
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  1%|          | 5/682 [00:00<00:16, 41.22it/s]

  2%|▏         | 11/682 [00:00<00:15, 44.42it/s]

  2%|▏         | 17/682 [00:00<00:13, 47.96it/s]

  3%|▎         | 23/682 [00:00<00:13, 50.64it/s]

  4%|▍         | 29/682 [00:00<00:12, 52.33it/s]

  5%|▌         | 35/682 [00:00<00:12, 52.83it/s]

  6%|▌         | 41/682 [00:00<00:12, 52.66it/s]

  7%|▋         | 46/682 [00:00<00:12, 49.00it/s]

  7%|▋         | 51/682 [00:01<00:17, 35.73it/s]

  8%|▊         | 55/682 [00:01<00:21, 29.86it/s]

  9%|▊         | 59/682 [00:01<00:22, 27.65it/s]

  9%|▉         | 63/682 [00:01<00:22, 27.77it/s]

 10%|▉         | 67/682 [00:01<00:24, 25.43it/s]

 10%|█         | 70/682 [00:01<00:23, 25.93it/s]

 11%|█         | 73/682 [00:02<00:24, 24.56it/s]

 11%|█         | 76/682 [00:02<00:24, 24.71it/s]

 12%|█▏        | 79/682 [00:02<00:25, 23.99it/s]

 12%|█▏        | 82/682 [00:02<00:24, 24.09it/s]

 12%|█▏        | 85/682 [00:02<00:25, 23.50it/s]

 13%|█▎ 

 71%|███████   | 481/682 [00:18<00:07, 27.01it/s]

 71%|███████   | 484/682 [00:18<00:07, 27.67it/s]

 71%|███████▏  | 487/682 [00:18<00:07, 27.11it/s]

 72%|███████▏  | 491/682 [00:18<00:07, 27.25it/s]

 72%|███████▏  | 494/682 [00:18<00:06, 27.85it/s]

 73%|███████▎  | 497/682 [00:18<00:07, 25.63it/s]

 73%|███████▎  | 500/682 [00:18<00:07, 24.96it/s]

 74%|███████▍  | 503/682 [00:18<00:07, 24.83it/s]

 74%|███████▍  | 507/682 [00:19<00:06, 25.66it/s]

 75%|███████▍  | 510/682 [00:19<00:07, 24.46it/s]

 75%|███████▌  | 513/682 [00:19<00:06, 24.78it/s]

 76%|███████▌  | 516/682 [00:19<00:06, 24.13it/s]

 76%|███████▌  | 519/682 [00:19<00:06, 23.63it/s]

 77%|███████▋  | 522/682 [00:19<00:06, 22.97it/s]

 77%|███████▋  | 525/682 [00:19<00:06, 23.13it/s]

 77%|███████▋  | 528/682 [00:19<00:06, 24.66it/s]

 78%|███████▊  | 531/682 [00:20<00:06, 24.86it/s]

 78%|███████▊  | 534/682 [00:20<00:05, 25.45it/s]

 79%|███████▊  | 537/682 [00:20<00:05, 24.42it/s]

 79%|███████▉  | 540/682 [00:20

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 8.92 s, sys: 585 ms, total: 9.51 s
Wall time: 26.7 s


In [94]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [46]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
lineFinder = LaneLineFinder()
challenge_clip = clip3.fl_image(lineFinder.process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4




  0%|          | 0/251 [00:00<?, ?it/s]

  2%|▏         | 4/251 [00:00<00:07, 31.11it/s]

  3%|▎         | 8/251 [00:00<00:07, 32.39it/s]

  5%|▍         | 12/251 [00:00<00:07, 32.53it/s]

  7%|▋         | 17/251 [00:00<00:06, 34.84it/s]

  9%|▉         | 22/251 [00:00<00:06, 36.61it/s]

 10%|█         | 26/251 [00:00<00:06, 37.14it/s]

 12%|█▏        | 30/251 [00:00<00:05, 37.14it/s]

 14%|█▎        | 34/251 [00:00<00:05, 36.86it/s]

 15%|█▌        | 38/251 [00:01<00:05, 36.49it/s]

 17%|█▋        | 42/251 [00:01<00:05, 35.89it/s]

 18%|█▊        | 46/251 [00:01<00:06, 33.58it/s]

 20%|█▉        | 50/251 [00:01<00:10, 19.19it/s]

 21%|██        | 53/251 [00:01<00:10, 19.24it/s]

 22%|██▏       | 56/251 [00:02<00:11, 16.52it/s]

 24%|██▎       | 59/251 [00:02<00:11, 16.32it/s]

 24%|██▍       | 61/251 [00:02<00:12, 15.54it/s]

 25%|██▌       | 63/251 [00:02<00:13, 14.39it/s]

 26%|██▌       | 65/251 [00:02<00:12, 15.45it/s]

 27%|██▋       | 67/251 [00:02<00:12, 14.21it/s]

 28%|██▊ 

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 5.19 s, sys: 430 ms, total: 5.62 s
Wall time: 19.9 s


In [47]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))